<a href="https://colab.research.google.com/github/gautamraj8044/UPI-Transaction-Extractor/blob/master/OCR_assigment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import re
import json
from paddleocr import PaddleOCR

# Initialize PaddleOCR model with English language support
ocr = PaddleOCR(use_angle_cls=True, lang='en')

amount_regex = re.compile(r'₹?\s?\d+(\.\d+)?|(\d+)\s?')
date_regex = re.compile(r'(\d{1,2}\s\w+\s\d{4})')
time_regex = re.compile(r'(\d{1,2}:\d{2}\s(?:AM|PM))')
upi_id_regex = re.compile(r'\b[A-Za-z0-9.]+@[a-z]+\b')
to_regex = re.compile(r'To:\s*([A-Za-z\s]+)(?:\s+UPI ID:)?')
from_regex = re.compile(r'From:\s*([A-Za-z\s]+)')

/usr/local/lib/python3.10/dist-packages/setuptools/command/easy_install.py:41: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_name

download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 4.00M/4.00M [00:18<00:00, 222kiB/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv3_rec_infer/en_PP-OCRv3_rec_infer.tar


100%|██████████| 9.96M/9.96M [00:19<00:00, 517kiB/s] 


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:15<00:00, 142kiB/s]


[2024/12/04 18:10:32] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_char_dict_path='/usr/local

In [2]:
#Load and preprocess the image for OCR.
def preprocess_image(image_path):
    # Load the image in color mode
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    if img is None:
        raise ValueError("Image not loaded correctly, please check the file path.")

    # Resize the image to a uniform size for OCR performance
    img = cv2.resize(img, (800, 1024))

    # Apply denoising to the image for better OCR performance
    img = cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 21)

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply adaptive thresholding to create a binary image
    binary_image = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 9, 3)

    return binary_image


In [3]:
def extract_text(image_path):
    result = ocr.ocr(image_path)
    extracted_lines = []

    for line in result:
        for word_info in line:
            extracted_lines.append(word_info[1][0])
    print(extracted_lines)
    return extracted_lines


In [4]:
def parse_details(extracted_lines):
    details = {}
    combined_text = '\n'.join(extracted_lines)
    transaction_status = re.search(r'(Paid Successfully|Failed)', combined_text)

    for line in extracted_lines:
        if line.isdigit():
            details['amount'] = line.strip()

        date_match = date_regex.search(line)
        time_match = time_regex.search(line)
        upi_id = upi_id_regex.search(line)
        to_match = to_regex.search(line)
        from_match = from_regex.search(line)
        upi_type_match = re.search(r'(Paytm|Google Pay|PhonePe)', line,re.IGNORECASE)

        if date_match:
            details['date'] = date_match.group(0).strip()
        if time_match:
            details['time'] = time_match.group(0).strip()
        if upi_id:
            details['UPI_ID'] = upi_id.group(0).strip()
        if to_match:
            details['To'] = to_match.group(1).strip()
        if from_match:
            details['From'] = from_match.group(1).strip()
        if upi_type_match:
            details['UPI_type'] = upi_type_match.group(0).strip()

    details['transaction_status'] = transaction_status.group(0) if transaction_status else 'Failed'

    return details


In [5]:
def structure_data(details):
    return {
        "transaction_status": details.get('transaction_status', 'N/A'),
        "amount": details.get('amount', 'N/A'),
        "date": details.get('date', 'N/A'),
        "time": details.get('time', 'N/A'),
        "UPI type": details.get('UPI_type', 'N/A'),
        "UPI ID": details.get('UPI_ID', 'N/A'),
        "To": details.get('To', 'N/A'),
        "From": details.get('From', 'N/A')
    }



In [6]:
def save_json(data, filename):
    with open(filename, 'w') as json_file:
        json.dump(data, json_file, indent=4)

In [7]:
def main(image_path):
    try:
        processed_image_path = "processed_image.jpg"
        image = preprocess_image(image_path)
        cv2.imwrite(processed_image_path, image)

        extracted_lines = extract_text(processed_image_path)
        parsed_details = parse_details(extracted_lines)
        structured_data = structure_data(parsed_details)
        print("Structured Data:\n", structured_data)
        json_filename = "transaction_details.json"
        save_json(structured_data, json_filename)
    except ValueError as e:
        print(e)

image_path = 'upiss.jpg'
main(image_path)

[2024/12/04 18:11:44] ppocr DEBUG: dt_boxes num : 14, elapse : 0.851163387298584
[2024/12/04 18:11:44] ppocr DEBUG: cls num  : 14, elapse : 0.17275333404541016
[2024/12/04 18:11:48] ppocr DEBUG: rec_res num  : 14, elapse : 3.8541176319122314
['Paytm', 'Paid Successfully', '3120', 'Rupees One Hundred Twenty Only', 'y Pay to BharatPe Merchant', 'To: Mr Devrai Rathore', 'UPI ID: BHARATPE', '.90063239027@fbpe', 'From: Gautam Raj', 'Punjab National Bank - 765o', 'UPl Ref No: 3254320307i7', 'O6:59 PM.11 Sep 2023', 'LFIIYES BANK']
Structured Data:
 {'transaction_status': 'Paid Successfully', 'amount': '3120', 'date': '11 Sep 2023', 'time': '6:59 PM', 'UPI type': 'Paytm', 'UPI ID': '90063239027@fbpe', 'To': 'Mr Devrai Rathore', 'From': 'Gautam Raj'}
